In [7]:
"""
data_partitioner.ipynb
Created by Chikuma, 08/22/2019
"""
import numpy as np
import pandas as pd
from math import floor

In [8]:
# read the data and show the data a bit
originalDataframe = pd.read_csv("../Dataset/TrainingSet.csv")
originalDataframe.describe()

,Entropy,Variance,RMS,PP,Energy,BSF,BPFI,BPFO,23 Hz
count,2048.000000,2048.000000,2048.000000,2048.000000,2.048000e+03,2048.000000,2048.000000,2048.000000,2048.000000
mean,6.024781,10368.341587,91.229060,2603.767188,7.349025e+11,0.152326,0.163637,0.112275,0.163637
std,0.532248,10741.483861,51.280495,1968.876782,7.212584e+11,0.105076,0.131028,0.110026,0.131028
min,5.127000,688.320000,26.298000,371.200000,4.640000e+10,0.002227,0.000535,0.001608,0.000535
25%,5.607325,1764.150000,42.075250,641.600000,1.190000e+11,0.056950,0.040418,0.033465,0.040418
50%,5.966600,6247.950000,83.455000,2367.200000,4.675000e+11,0.129545,0.111100,0.062867,0.111100
75%,6.521700,16586.000000,130.280000,3512.000000,1.140000e+12,0.242155,0.279273,0.160303,0.279273
max,6.906200,39762.000000,199.460000,9760.000000,2.670000e+12,0.406990,0.424120,0.420100,0.424120


In [9]:
# add label to the data (256 per condition)
label = [floor(i / 256) for i in range(2048)]
originalDataframe["Condition"] = label

# sneak a peek of what you have done :)
originalDataframe.describe()

,Entropy,Variance,RMS,PP,Energy,BSF,BPFI,BPFO,23 Hz,Condition
count,2048.000000,2048.000000,2048.000000,2048.000000,2.048000e+03,2048.000000,2048.000000,2048.000000,2048.000000,2048.000000
mean,6.024781,10368.341587,91.229060,2603.767188,7.349025e+11,0.152326,0.163637,0.112275,0.163637,3.500000
std,0.532248,10741.483861,51.280495,1968.876782,7.212584e+11,0.105076,0.131028,0.110026,0.131028,2.291847
min,5.127000,688.320000,26.298000,371.200000,4.640000e+10,0.002227,0.000535,0.001608,0.000535,0.000000
25%,5.607325,1764.150000,42.075250,641.600000,1.190000e+11,0.056950,0.040418,0.033465,0.040418,1.750000
50%,5.966600,6247.950000,83.455000,2367.200000,4.675000e+11,0.129545,0.111100,0.062867,0.111100,3.500000
75%,6.521700,16586.000000,130.280000,3512.000000,1.140000e+12,0.242155,0.279273,0.160303,0.279273,5.250000
max,6.906200,39762.000000,199.460000,9760.000000,2.670000e+12,0.406990,0.424120,0.420100,0.424120,7.000000


In [24]:
# feed into a condition, get training and validation data of K
def seperateTrainValidation(dataframe, kPart, K):
    partitionSize = int(len(dataframe) / K)
    partitions = [dataframe.iloc[partitionSize * i: partitionSize * (i + 1)] for i in range(K)]
    
    validationData = partitions[kPart]
    trainingData = pd.DataFrame(columns = dataframe.columns)
    for i in range(K):
        if i is not kPart:
            trainingData = trainingData.append(partitions[i], ignore_index = True)

    return trainingData, validationData

In [25]:
# split the data by conditions
originalDataframeSize = int(len(originalDataframe) / 8)
conditions = [originalDataframe.iloc[originalDataframeSize * i: originalDataframeSize * (i + 1)] for i in range(8)]

In [32]:
K = 4
part = 3

trainingData = pd.DataFrame(columns = originalDataframe.columns)
validationData = pd.DataFrame(columns = originalDataframe.columns)

for i in range(8):
    train, val = seperateTrainValidation(conditions[i], part, K)
    trainingData = trainingData.append(train)
    validationData = validationData.append(val)
    
trainingData.to_csv("../Dataset/trainingData_{}.csv".format(part), index = False)
validationData.to_csv("../Dataset/validationData_{}.csv".format(part), index = False)

In [28]:
trainingData

,Entropy,Variance,RMS,PP,Energy,BSF,BPFI,BPFO,23 Hz,Condition
0,6.0365,726.43,27.012,491.2,4.900000e+10,0.025956,0.026630,0.018925,0.026630,0
1,6.0389,727.36,27.124,484.8,4.940000e+10,0.038400,0.020192,0.027468,0.020192,0
2,6.0773,763.81,27.640,483.2,5.130000e+10,0.066190,0.002209,0.020683,0.002209,0
3,6.0726,748.95,27.371,432.0,5.030000e+10,0.029492,0.063825,0.025780,0.063825,0
4,6.0483,724.15,27.012,472.0,4.900000e+10,0.015728,0.030236,0.034100,0.030236,0
5,6.0590,747.51,27.368,507.2,5.030000e+10,0.077257,0.029326,0.011524,0.029326,0
6,6.0438,721.54,26.874,480.0,4.850000e+10,0.023505,0.031963,0.025671,0.031963,0
7,6.0627,743.29,27.289,460.8,5.000000e+10,0.011697,0.022466,0.028103,0.022466,0
8,6.0715,743.09,27.278,419.2,4.990000e+10,0.082963,0.050477,0.065647,0.050477,0
9,6.0498,733.61,27.208,419.2,4.970000e+10,0.034394,0.021873,0.025050,0.021873,0
